In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np

# 加载文档内容（每行为一个 chunk）
with open("data/test.txt", "r", encoding="utf-8") as f:
    docs = [line.strip() for line in f if line.strip()]

# 加载 embedding 模型（可替换为 bge/text2vec）
model = SentenceTransformer("all-mpnet-base-v2")

print(docs)

# 生成向量（默认维度 384）
embeddings = model.encode(docs, normalize_embeddings=True)
print(f"共生成 {len(embeddings)} 个向量，每个维度为 {embeddings.shape[1]}")


/Users/bierxiensi/Desktop/flood-rag/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/bierxiensi/Desktop/flood-rag/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['第七条：公司是企业法人，有独立的法人财产。', '第八条：公司以其全部财产对公司债务承担责任。', '董事会对公司的经营决策具有最终权力。', '股东大会是公司的最高权力机构。', '李彦宏称大模型成本每年降低90%。', '乌军大批直升机击落多架俄无人机。', '王力宏回应是否想找新伴侣。', '饺子不知道观众怎么想出的藕饼cp。', '加沙停火协议关键时刻生变。', 'The weather is lovely today。', "It's so sunny outside。", 'He drove to the stadium。']
共生成 12 个向量，每个维度为 768


In [2]:
import faiss

# 创建 IndexFlatIP（使用 Cosine 相似度需先归一化）
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)
print(f"向量索引中共有 {index.ntotal} 个向量")

向量索引中共有 12 个向量


In [4]:
query = "国际新闻"
query_vec = model.encode([query], normalize_embeddings=True)

print(f"查询向量维度为 {query_vec.shape}")

# 查询最相似的 top-k 向量
k = 10
D, I = index.search(np.array(query_vec), k)

# 输出检索结果
for rank, idx in enumerate(I[0]):
    print(f"[Top {rank+1}] {docs[idx]} 相似度: {D[0][rank]:.4f}")


查询向量维度为 (1, 768)
[Top 1] 王力宏回应是否想找新伴侣。 相似度: 0.7094
[Top 2] 第七条：公司是企业法人，有独立的法人财产。 相似度: 0.5587
[Top 3] 股东大会是公司的最高权力机构。 相似度: 0.5398
[Top 4] 乌军大批直升机击落多架俄无人机。 相似度: 0.5285
[Top 5] 李彦宏称大模型成本每年降低90%。 相似度: 0.5203
[Top 6] 第八条：公司以其全部财产对公司债务承担责任。 相似度: 0.5114
[Top 7] 董事会对公司的经营决策具有最终权力。 相似度: 0.5025
[Top 8] 加沙停火协议关键时刻生变。 相似度: 0.4822
[Top 9] 饺子不知道观众怎么想出的藕饼cp。 相似度: 0.4807
[Top 10] It's so sunny outside。 相似度: 0.2071
